In [1]:
with open('input.txt', 'r') as file:
    text = file.read()

In [2]:
chars = sorted(list(set(text)))
vocab_size = len(chars)
print(''.join(chars))
print(vocab_size)


 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
65


In [3]:
stoi = {s:i for i,s in enumerate(chars)}
itos = {i:s for i,s in enumerate(chars)}
encode = lambda s: [stoi[c] for c in s]
decode = lambda l: ''.join([itos[i] for i in l])

print(encode('hello there'))
print(decode(encode('hello there')))

[46, 43, 50, 50, 53, 1, 58, 46, 43, 56, 43]
hello there


In [4]:
import torch
data = torch.tensor(encode(text), dtype=torch.long)

In [5]:
n = int(0.9*len(data))
train_data = data[:n]
val_data = data[n:]

In [6]:
block_size = 8
x = train_data[:block_size].tolist()
y = train_data[1:block_size+1].tolist()
for t in range(block_size):
    context = x[:t+1]
    target = y[t]
    print(f"when the input is {context} the target is {target}")

when the input is [18] the target is 47
when the input is [18, 47] the target is 56
when the input is [18, 47, 56] the target is 57
when the input is [18, 47, 56, 57] the target is 58
when the input is [18, 47, 56, 57, 58] the target is 1
when the input is [18, 47, 56, 57, 58, 1] the target is 15
when the input is [18, 47, 56, 57, 58, 1, 15] the target is 47
when the input is [18, 47, 56, 57, 58, 1, 15, 47] the target is 58


In [7]:
torch.manual_seed(1337)
batch_size = 4
block_size = 8

def get_batch(split):
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data)- block_size, (batch_size,))
    x = torch.stack([data[i:block_size+i] for i in ix])
    y = torch.stack([data[i+1: i+block_size+1] for i in ix])
    return x, y

xb, yb = get_batch('train')

In [37]:
import torch
import torch.nn as nn
import torch.nn.functional as F
torch.manual_seed(1337)

class BigramLanguageModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)

    def forward(self, idx, targets = None):
        logits = self.token_embedding_table(idx)
        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)
        
        return logits, loss

    def generate(self, idx, max_new_tokens):
        for _ in range(max_new_tokens):
            logits, loss = self(idx)
            logits = logits[:, -1, :]
            probs = F.softmax(logits, dim=-1)
            idx_next = torch.multinomial(probs, num_samples = 1)
            idx = torch.cat((idx, idx_next), dim=1)
        return idx

    

m = BigramLanguageModel(vocab_size)
logits, loss = m(xb, yb)

print(decode(m.generate(torch.zeros((1,1), dtype = torch.long), max_new_tokens = 100)[0].tolist()))


Sr?qP-QWktXoL&jLDJgOLVz'RIoDqHdhsV&vLLxatjscMpwLERSPyao.qfzs$Ys$zF-w,;eEkzxjgCKFChs!iWW.ObzDnxA Ms$3


In [38]:
optimizer = torch.optim.AdamW(m.parameters(), lr=1e-3)

In [44]:
batch_size = 32
for steps in range(1000):
    xb, yb = get_batch('train')
    logits, loss = m(xb, yb)
    
    optimizer.zero_grad(set_to_none = True)
    loss.backward()
    optimizer.step()

print(loss.item())


2.632219076156616


In [46]:
print(decode(m.generate(torch.zeros((1,1), dtype = torch.long), max_new_tokens = 100)[0].tolist()))


A::'
t.
Sin honk$zlerseestindovrer wat boue nodgh flle, mOM:

SCKpoto AKIFwinthind d me sesete gkerw


In [61]:
torch.manual_seed(1337)
B,T,C = 4, 8, 32
x = torch.randn(B,T,C)

head_size = 16
key 

tril = torch.tril(torch.ones(T, T))
wei = torch.ones((T, T))
wei = wei.masked_fill(tril == 0, float('-inf'))
wei = F.softmax(wei, dim=-1)
out = wei @ x